# Roadmap

1. Get data
2. Create train-test split
3. Clean training set (write functions): missing values, text, categorical attributes, scaling
4. Select models and scoring metrics, then train
5. Compare them: clean test set, make predictions, score
6. Fine-tune models

## 1. Load data

In [1]:
from functions import load_data
data_df = load_data('data/time_series.xlsx')
data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 292 entries, 2012-04-08 to 2017-11-05
Columns: 1833 entries, 012 to TRUHONE
dtypes: int64(1833)
memory usage: 4.1 MB


## 2. Train-Test Split

In [2]:
# splitting into training and testing sets setting aside last year for testing
from functions import ts_train_test_split

train_df, test_df = ts_train_test_split(data_df, 52)
train_df.tail()

Observations: 292
Training Observations: 240
Testing Observations: 52


,012,017,03008944ST-1,03008944ST-3,0300ST1550-1,0300ST15X9-1,0300ST15X9-2,0300ST15X9-3,0300ST1605-1,0300ST1605-2,...,9920-2,9920-3,9920-4,9920-5,9920-6,9920-7,9997-25,HW220D15,HW240DIA,TRUHONE
EntDate,,,,,,,,,,,,,,,,,,,,,
2016-10-09,0,0,0,0,0,0,0,0,0,0,...,12,9,0,0,6,0,0,0,0,0
2016-10-16,0,0,0,0,0,0,0,0,0,0,...,31,4,5,4,15,2,0,0,0,0
2016-10-23,0,0,0,0,0,0,0,0,0,0,...,158,27,3,293,199,0,5,0,0,0
2016-10-30,0,0,0,0,0,0,0,0,0,0,...,23,4,0,221,47,6,1,0,0,0
2016-11-06,0,0,0,0,0,0,0,0,0,0,...,12,18,8,0,42,1,0,0,0,0


In [ ]:
# DO NOT TOUCH Test Set!!!! Data snooping no-no

## Make Groups

In [3]:
# make a list of column names
product_SKUs = list(train_df.columns.values)
train_df.tail(52)
# 2015-11-15 is exactly one year
len(product_SKUs)

1833

In [17]:
# non_active products
# defined as products that have not moved in n_weeks time periods
# we will start with n_weeks 52 (1 year)

from datetime import datetime

def identify_non_active(dataframe, product_list, year, month, day):
    last_tp = (dataframe[datetime(year, month, day):])

    non_active = []
    for product in product_list:
        if last_tp[product].sum() == 0:
            non_active.append(product)
    return non_active

non_active = identify_non_active(train_df, product_SKUs, 2015, 11, 4)
len(non_active)

472

In [18]:
# 1833 products, 472 non_active, leaving 1361, new?
len(product_SKUs) - len(non_active)

1361

In [20]:
# new products 
train_df[:datetime(2014, 5, 3)]

def identify_new_product(dataframe, product_list, year, month, day):
    previous_tp = (dataframe[:datetime(year, month, day)])
    last_tp = (dataframe[datetime(year, month, day):])

    new_products = []
    for product in product_list:
        if previous_tp[product].sum() == 0 and last_tp[product].sum() !=0:
            new_products.append(product)
    return new_products
new_products = identify_new_product(train_df, product_SKUs, 2015, 11, 4)
len(new_products)

138

In [21]:
# 1833 products, 472 non_active, 138 new, leaving 1223, new?
len(product_SKUs) - len(non_active) - len(new_products)

1223

In [87]:
last_tp = train_df.tail(12)
intermittent = product_SKUs.copy()
for product in product_SKUs:
    if last_tp[product].sum() == 0:
        intermittent.remove(product)
len(intermittent)

1112

In [95]:
# intermittent demand
# iterate over last year, if n consecutive values = 0, but sum != 0 
# (they are all active so no need to check), it's intermittent
def identify_intermittent_product(dataframe, product_list, year, month, day, n):
    last_tp = (dataframe[datetime(year, month, day):])

    intermittent = product_list.copy()
    active = []
    for product in product_list:
        if last_tp[product].sum() != 0:
            active.append(product)
    
    for product in active:
        if last_tp[product].rolling(n).sum().dropna().nonzero():
            intermittent.remove(product)
    return intermittent
    
my_list = identify_intermittent_product(train_df, product_SKUs, 2015, 11, 4, 4)
len(my_list)

472

In [39]:
testing = train_df[0:12]
testing

,012,017,03008944ST-1,03008944ST-3,0300ST1550-1,0300ST15X9-1,0300ST15X9-2,0300ST15X9-3,0300ST1605-1,0300ST1605-2,...,9920-2,9920-3,9920-4,9920-5,9920-6,9920-7,9997-25,HW220D15,HW240DIA,TRUHONE
EntDate,,,,,,,,,,,,,,,,,,,,,
2012-04-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2012-04-15,0,0,0,0,0,0,0,0,0,0,...,20,19,7,166,98,0,0,0,0,0
2012-04-22,0,0,0,0,0,0,0,0,0,0,...,41,15,10,207,87,0,0,0,0,0
2012-04-29,0,0,0,0,0,0,0,0,0,0,...,38,44,13,101,21,10,0,0,0,0
2012-05-06,0,0,0,0,0,0,0,0,0,0,...,29,47,21,75,43,0,0,0,0,0
2012-05-13,0,0,0,0,0,0,0,0,0,0,...,18,35,0,133,34,0,0,0,0,0
2012-05-20,0,0,0,0,0,0,0,0,0,0,...,42,24,0,90,28,0,0,0,0,0
2012-05-27,0,0,0,0,0,0,0,0,0,0,...,6,20,6,124,73,0,0,0,0,1
2012-06-03,0,0,0,0,0,0,0,0,0,0,...,52,36,13,177,26,0,0,0,0,0


In [66]:
testing2 = testing["TRUHONE"].rolling(5).sum()
testing2

EntDate
2012-04-08    NaN
2012-04-15    NaN
2012-04-22    NaN
2012-04-29    NaN
2012-05-06    0.0
2012-05-13    0.0
2012-05-20    0.0
2012-05-27    1.0
2012-06-03    1.0
2012-06-10    1.0
2012-06-17    1.0
2012-06-24    1.0
Name: TRUHONE, dtype: float64

In [70]:
testing3 = testing2.dropna()
testing3

EntDate
2012-05-06    0.0
2012-05-13    0.0
2012-05-20    0.0
2012-05-27    1.0
2012-06-03    1.0
2012-06-10    1.0
2012-06-17    1.0
2012-06-24    1.0
Name: TRUHONE, dtype: float64

In [77]:
testing4 = testing["TRUHONE"].rolling(5).sum().dropna()
if testing4.nonzero():
    print('yay')

yay


In [67]:
testing3 = testing2.drop(testing2.index[:4])
testing3

EntDate
2012-05-06    0.0
2012-05-13    0.0
2012-05-20    0.0
2012-05-27    1.0
2012-06-03    1.0
2012-06-10    1.0
2012-06-17    1.0
2012-06-24    1.0
Name: TRUHONE, dtype: float64

In [60]:
if testing["TRUHONE"].rolling(5).sum().isnull():
    print('good')

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [50]:
intermittent = []
for product in product_SKUs:
    if testing[product].rolling(n).sum() == 0:
        intermittent.append(product)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [44]:
testing['9920-2'].rolling(3).sum()

EntDate
2012-04-08      NaN
2012-04-15      NaN
2012-04-22     61.0
2012-04-29     99.0
2012-05-06    108.0
2012-05-13     85.0
2012-05-20     89.0
2012-05-27     66.0
2012-06-03    100.0
2012-06-10     70.0
2012-06-17     77.0
2012-06-24     47.0
Name: 9920-2, dtype: float64

## Select single item for forecasting

In [ ]:
# pick one item
test = train_df['03108627CC']
test.head()

In [ ]:
# plot one item
from functions import plot_train_test
plot_train_test(train_df, test_df, '03108627CC')

## 3. Clean Training Set

In [ ]:
# make a copy first

In [ ]:
# make transformations pipeline (first applied to train, then apply to test)
# luckily, our data has no null values, no categorical/text values
# however, to get the initial messy excel sheet into timeseries format was not easy
# describe

## 4. Select Models and Scoring Metrics

We are comparing different forecasting models on the same data, so we can use scale-dependent errors because our single dataset only has one scale. We will use two metrics: <br>
* MAD (also called MAE): $\frac{|A_t-F_t|}{n}$ <br>
* RMSE (root mean squared error): $\sqrt{\frac{(A_t-F_t)^2}{n}}$ <br>

For baseline, we will use:
* naive forecast ("only yesterday matters")

Then we will try two simple forecasts:
* cumulative ("history matters")
* moving average ("I select how much matters")

We will also try:
* ARIMA
* exponential smoothing

### Make Forecasts

In [ ]:
# make a df to store all our predictions
from functions import make_copy_df
y_hat = make_copy_df(test_df, '9920-2')

### Naive Forecast (Baseline)

In [ ]:
# FIX LATER
import numpy as np

dd= np.asarray(train_df['9920-2'])
y_hat['naive'] = dd[len(dd)-1] # this line of code is for one-time forecast

In [ ]:
# plot
from functions import plot_time_series
plot_time_series(train_df, test_df, '9920-2', y_hat, 'naive', 'Naive Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'naive')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df, '9920-2', y_hat, 'naive')

### Cumulative

In [ ]:
from functions import cumulative
y_hat['cumulative'] = cumulative(train_df['9920-2'])
plot_time_series(train_df, test_df, '9920-2', y_hat, 'cumulative', 'Cumulative Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'cumulative')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'cumulative')

### Moving Average

In [ ]:
# forecasts for a quarter year do better than half year or full year
from functions import moving_average
y_hat['moving_avg'] = moving_average(train_df['9920-2'], m=13)

plot_time_series(train_df, test_df, '9920-2', y_hat, 'moving_avg', 'Moving Average Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'moving_avg')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'moving_avg')

### S/ARIMA

In [ ]:
# gridsearch with pyramid
from pyramid.arima import ARIMA

fit = ARIMA(order=(1, 1, 0), seasonal_order=(1, 0, 0, 12)).fit(y=train_df['9920-2'])

from pyramid.arima import auto_arima

stepwise_fit = auto_arima(train_df['9920-2'], start_p=0, start_q=0, max_p=6, max_q=6, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
# fit with SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX

fit1 = SARIMAX(train_df['9920-2'], order=(0, 1, 1), seasonal_order=(0,1,2,12), freq='W').fit()
y_hat['SARIMA'] = fit1.predict(start="2016-11-06", end="2017-11-05", dynamic=True, typ='levels')

# plot
plot_time_series(train_df, test_df, '9920-2', y_hat, 'SARIMA', 'SARIMA Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'SARIMA')

### Exponential Smoothing

In [ ]:
# let's decompose it first to see trend, seasonality
from functions import decompose_timeseries

decompose_timeseries(train_df['9920-2'], 'additive')

In [ ]:
# Double Exponential Smoothing with Statsmodels
# no trend, just seasonality (multiplicative), no damping
from statsmodels.tsa.api import ExponentialSmoothing

fit1 = ExponentialSmoothing(np.asarray(train_df['9920-2']), seasonal_periods=10, trend=None, seasonal='additive').fit(smoothing_level=0.51, smoothing_seasonal=0.1)
y_hat['DES'] = fit1.forecast(len(test_df))

# plot
plot_time_series(train_df, test_df, '9920-2', y_hat, 'DES', 'Double ES Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'DES')

In [ ]:
# here we have trend and seasonality, so we will use Holt-Winters with Statsmodels
# additive trend and seasonality, no trend damping, seasonal periods=10
from statsmodels.tsa.api import ExponentialSmoothing

fit1 = ExponentialSmoothing(np.asarray(train_df['9920-2']), seasonal_periods=10, trend='add', seasonal='add').fit(smoothing_level=0.51, smoothing_slope=0.015,smoothing_seasonal=0.1)
y_hat['Holt_Winter'] = fit1.forecast(len(test_df))

plot_time_series(train_df, test_df, '9920-2', y_hat, 'Holt_Winter', 'Holt_Winter Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df,'9920-2', y_hat, 'Holt_Winter')

In [ ]:
# calculate MAE
from functions import MAE

MAE(test_df,'9920-2', y_hat, 'Holt_Winter')

### Exponential Smoothing with Homebrewed Functions

In [ ]:
from exponential_smoothing import initial_trend, initial_seasonal_components, triple_exponential_smoothing

In [ ]:
predictions = triple_exponential_smoothing(train_df['9920-2'], slen=10, alpha=0.51, beta=0.015, gamma=0.1, n_preds=52)
y_hat['HW_new'] = predictions[-52:]

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', y_hat, 'HW_new')

### Exponential Smoothing with Homebrewed Class

In [ ]:
# additive trend and seasonality, no trend damping, seasonal periods=10
from HoltWinters_class2 import HoltWinters

In [ ]:
model = HoltWinters(alpha=0.51, beta=0.015, gamma=0.1, n_preds=52)
model.fit(train_df['9920-2'])

preds = model.predict(train_df['9920-2'])
y_hat['Holt_Winter'] = preds[-52:]
plot_time_series(train_df, test_df, '9920-2', y_hat, 'Holt_Winter', 'Holt_Winter Forecast')

In [ ]:
# calculate RMSE
model.score(train_df['9920-2'], test_df['9920-2'])

In [ ]:
# gridsearch
from sklearn.model_selection import GridSearchCV
from HoltWinters_class2 import HoltWinters

# create a parameter grid: map the parameter names to the values
param_grid = {'alpha': [0.02, 0.1, 0.19, 0.35, 0.51],                   
                'beta': [0.005, 0.029, 0.053, 0.094, 0.135, 0.176],
                'gamma': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]}
# instantiate the grid
grid_search = GridSearchCV(HoltWinters(), param_grid)
# fit the grid with data
grid_search.fit(train_df['9920-2']).score(X=train_df['9920-2'], y=test_df['9920-2'])
# print best results
grid_search.best_params_ 

In [ ]:
# create a parameter grid: map the parameter names to the values
param_grid = {'alpha': [0.02, 0.1, 0.19, 0.35, 0.51],                   
                'beta': [0.005, 0.029, 0.053, 0.094, 0.135, 0.176],
                'gamma': [0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
              'slen': [10],
              'n_preds': [100]}

### Prophet

In [ ]:
# imports 
from fbprophet import Prophet

In [ ]:
# make a copy of the dataframe for Prophet transformations
from functions import make_copy_df
prophet_df = make_copy_df(train_df, '9920-2')

In [ ]:
# rename variables (prophet requires the variable names in the time series to be
# y for target and ds for Datetime)
from functions import rename_columns
rename_columns(prophet_df, '9920-2')
prophet_df.head()

In [ ]:
# FIX LATER
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(growth='linear', interval_width=0.95, weekly_seasonality=True)
my_model.fit(prophet_df)

In [ ]:
# FIX LATER
future_dates = my_model.make_future_dataframe(periods=52, freq='W')
forecast = my_model.predict(future_dates)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
# FIX LATER
forecast.head()
forecast = forecast.set_index('ds')
forecast_slice=forecast[240:292]
forecast_df = forecast_slice["yhat"]
forecast_df.head()

In [ ]:
# plot
plot_time_series(train_df, test_df, '9920-2', forecast_slice, 'yhat', 'Prophet Forecast')

In [ ]:
# calculate RMSE
from functions import RMSE

RMSE(test_df, '9920-2', forecast_slice, 'yhat')

## Resources

Great basic blogs on time series and forecasting methods: https://medium.com/@josemarcialportilla/using-python-and-auto-arima-to-forecast-seasonal-time-series-90877adff03c

On Prophet:
https://www.analyticsvidhya.com/blog/2018/05/generate-accurate-forecasts-facebook-prophet-python-r/
https://research.fb.com/prophet-forecasting-at-scale/
(Plus see 